<a href="https://colab.research.google.com/github/Dadoyen2/Geometry-Optimization/blob/main/geometry_optimization_in_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import requests
import math
def get_coordinate(url):
    atoms = []
    response = requests.get(url)
    response.raise_for_status()
    lines = response.text.splitlines()

    # Start parsing for atom data directly after identifying the pattern
    for line in lines:
        parts = line.split()
        # Check if line contains a potential atom data pattern
        if len(parts) >= 4 and parts[3].isalpha():
            try:
                # Attempt to parse the first three values as coordinates
                x, y, z = map(float, parts[:3])
                atom_type = parts[3]
                atoms.append((atom_type, x, y, z))
            except ValueError:
                # Skip lines that don't fit the pattern (ensures generality)
                continue

    return atoms

def print_atom_coordinates(atoms):
    print(f"The input file has: {len(atoms)} atoms")
    print("Atoms and coordinates (in Å):")
    for atom in atoms:
        print(f"{atom[0]:<2} {atom[1]:>8.4f} {atom[2]:>8.4f} {atom[3]:>8.4f}")

# Example URL to use with the general extraction function
url = 'https://raw.githubusercontent.com/Dadoyen2/Geometry-Optimization/main/ethane_dist.mol2'
atoms = get_coordinate(url)
print_atom_coordinates(atoms)




The input file has: 8 atoms
Atoms and coordinates (in Å):
C   -0.7560   0.0500   0.0000
C    0.7960   0.0000   0.0500
H   -1.1804   0.6486   0.7945
H   -1.1004   0.3601  -0.9626
H   -1.2405  -0.9987   0.4781
H    1.1204  -0.3401   0.9526
H    1.1605   1.0187  -0.1581
H    1.1904  -0.6786  -0.7745


In [52]:
import numpy as np
import requests

def bond_length(coord1, coord2):
    return np.linalg.norm(np.array(coord2) - np.array(coord1))

def bond_angle(coord1, coord2, coord3):
    vec1 = np.array(coord1) - np.array(coord2)
    vec2 = np.array(coord3) - np.array(coord2)
    cos_theta = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return np.degrees(np.arccos(cos_theta))



def dihedral_angle(coord1, coord2, coord3, coord4):
    # Define the vectors
    vec1 = np.array(coord1) - np.array(coord2)  # r_AB
    vec2 = np.array(coord3) - np.array(coord2)  # r_BC
    vec3 = np.array(coord4) - np.array(coord3)  # r_CD

    # Calculate normal vectors to planes
    t = np.cross(vec1, vec2)  # Normal to plane ABC
    u = np.cross(vec2, vec3)  # Normal to plane BCD

    # Calculate vector perpendicular to both normal vectors
    v = np.cross(t, u)

    # Compute cos(phi)
    cos_phi = np.dot(t, u) / (np.linalg.norm(t) * np.linalg.norm(u))

    # Compute sin(phi)
    sin_phi = np.dot(vec2 / np.linalg.norm(vec2), v) / (np.linalg.norm(t) * np.linalg.norm(u))

    # Correct numerical inaccuracies (clipping for cos_phi)
    cos_phi = np.clip(cos_phi, -1.0, 1.0)

    # Calculate dihedral angle
    angle_rad = np.arctan2(sin_phi, cos_phi)  # Angle in radians
    angle_degree = np.degrees(angle_rad)     # Convert radians to degrees

    # Normalize angle to the range 0° to 360°
    if angle_degree < 0:
        angle_degree += 360
    angle_rad = np.radians(angle_degree)

  #  return angle_degree, angle_rad

    # Subtract the angle from 180 degrees
    corrected_angle_degree = angle_degree -180
    corrected_angle_rad = np.radians(corrected_angle_degree)

    return corrected_angle_degree, corrected_angle_rad


# Stretching (Bond) Energy Calculation
    # Parameters for bond lengths and constants
bond_params = {
    ('C', 'H'): (350, 1.11),  # Example: k_b and r_0 for C-H bond
    ('C', 'C'): (300, 1.53),  # Example: k_b and r_0 for C-C bond
    #
}

# Define distance cutoffs to distinguish bonded vs non-bonded interactions
bond_cutoffs = {
    ('C', 'H'): (0.5, 1.5),  # Minimum and maximum cutoff for C-H bond detection
    ('C', 'C'): (1.0, 2.0),  # Minimum and maximum cutoff for C-C bond detection
    # Add other cutoffs as needed
}

# Stretching (Bond) Energy Calculation with flexibility for any C-H or hydrocarbon bonds
stretch_energy = 0.0
bond_count = 0
visited_bonds = set()  # To keep track of counted bonds
bonds_new = []
for i in range(len(atoms) - 1):
    for j in range(i + 1, len(atoms)):
        bond_type = tuple(sorted((atoms[i][0], atoms[j][0])))  # Determine bond type dynamically

        # Only consider bond types defined in bond_params and bond_cutoffs
        if bond_type in bond_params and bond_type in bond_cutoffs:
            # Calculate bond length
            r = bond_length(atoms[i][1:], atoms[j][1:])

            # Apply distance cutoff to avoid counting non-bonded pairs
            min_cutoff, max_cutoff = bond_cutoffs[bond_type]
            if min_cutoff <= r <= max_cutoff and (i, j) not in visited_bonds:
                visited_bonds.add((i, j))  # Mark this bond as visited

                # Get the bond parameters
                k_b, r_0 = bond_params[bond_type]
                bonds_new.append([i,j])
                # Calculate energy for this bond and add to total
                bond_energy = k_b * (r - r_0) ** 2
                stretch_energy += bond_energy
                bond_count += 1

                # Debugging output for verification
                print(f"Bond {bond_type[0]}-{bond_type[1]}: Length = {r:.2f} Å, Energy = {bond_energy:.4f} kcal/mol")

# Output the final results
print(f"\nTotal Stretching (Bond) Energy: {stretch_energy:.4f} kcal/mol")
print(f"Number of Bonds: {bond_count}")
print(r)



Bond C-C: Length = 1.55 Å, Energy = 0.1672 kcal/mol
Bond C-H: Length = 1.08 Å, Energy = 0.2841 kcal/mol
Bond C-H: Length = 1.07 Å, Energy = 0.6071 kcal/mol
Bond C-H: Length = 1.25 Å, Energy = 6.8832 kcal/mol
Bond C-H: Length = 1.02 Å, Energy = 2.9857 kcal/mol
Bond C-H: Length = 1.10 Å, Energy = 0.0237 kcal/mol
Bond C-H: Length = 1.14 Å, Energy = 0.2814 kcal/mol

Total Stretching (Bond) Energy: 11.2323 kcal/mol
Number of Bonds: 7
1.1383538861004516


In [53]:


# Bending energy calculation
bond_distance_threshold = 1.6  # Bonding threshold in angstroms
equilibrium_angle_degrees = 109.5  # Equilibrium bond angle for sp³ carbons

# Define k_a values for bending energy
ka_values_degree = {
    ('H', 'C', 'H'): 35,
    ('H', 'C', 'C'): 35,
    ('C', 'C', 'C'): 60
}

# Initialize bending energy calculations
bend_energy_degrees = 0.0
bend_energy_radians = 0.0
angle_count = 0
angles_new = []  # List to store angles for Wilson B matrix calculation

# Calculate angles and bending energy
for j in range(len(atoms)):
    if atoms[j][0] == 'C':  # Central atom must be carbon
        # Find atoms bonded to the central carbon atom
        bonded_atoms = [
            i for i in range(len(atoms))
            if i != j and bond_length(atoms[i][1:], atoms[j][1:]) <= bond_distance_threshold
        ]

        # Iterate over unique pairs of bonded atoms
        for m in range(len(bonded_atoms)):
            for n in range(m + 1, len(bonded_atoms)):
                i, k = bonded_atoms[m], bonded_atoms[n]

                # Determine the angle type and corresponding k_a value
                angle_type = (atoms[i][0], atoms[j][0], atoms[k][0])
                ka_degree = ka_values_degree.get(angle_type, 35)  # Default to 35 if not found
                angles_new.append([i, j, k])  # Store the angle triplet

                # Calculate bond angle in degrees
                angle_degrees = bond_angle(atoms[i][1:], atoms[j][1:], atoms[k][1:])

                # Calculate bending energy in degrees
                angle_energy_degrees = ka_degree * (angle_degrees - equilibrium_angle_degrees) ** 2
                bend_energy_degrees += angle_energy_degrees

                # Convert energy from degrees to radians
                conversion_factor = (np.pi / 180) ** 2
                angle_energy_radians = angle_energy_degrees * conversion_factor
                bend_energy_radians += angle_energy_radians

                # Increment angle count
                angle_count += 1

                # Print the results
                print(f"{atoms[i][0]}{i + 1} - {atoms[j][0]}{j + 1} - {atoms[k][0]}{k + 1}: "
                      f"{angle_degrees:>10.3f} {np.radians(angle_degrees):>10.3f} "
                      f"{angle_energy_degrees:>20.6f} {bend_energy_radians:>20.6f}")

# Print total bending energy
print(f"\nTotal bending energy in degrees: {bend_energy_degrees:.6f} kcal/mol")
print(f"Total bending energy in radians: {bend_energy_radians:.6f} kcal/mol")
print(f"Total number of angles calculated: {angle_count}")

# Angles list for Wilson B matrix
print("\nAngles list for Wilson B matrix:")
print(angles_new)



C2 - C1 - H3:    112.717      1.967           362.187201             0.110329
C2 - C1 - H4:    111.123      1.939            92.193260             0.138412
C2 - C1 - H5:    110.354      1.926            25.538209             0.146192
H3 - C1 - H4:    112.009      1.955           220.283660             0.213294
H3 - C1 - H5:     91.792      1.602         10975.255770             3.556548
H4 - C1 - H5:    117.587      2.052          2289.193178             4.253876
C1 - C2 - H6:    110.962      1.937            74.801757             4.276662
C1 - C2 - H7:    107.137      1.870           195.496814             4.336214
C1 - C2 - H8:    109.998      1.920             8.667359             4.338854
H6 - C2 - H7:    111.782      1.951           182.190328             4.394352
H6 - C2 - H8:    109.435      1.910             0.146832             4.394397
H7 - C2 - H8:    107.443      1.875           148.139085             4.439523

Total bending energy in degrees: 14574.093454 kcal/mol
Total be

In [54]:

# Torsion energy parameters
torsion_params = {
    "v_n": 0.3,  # kcal/mol, torsional barrier
    "n": 3,      # Periodicity of torsion angle
}

torsion_n = []  # To store torsion angles for later use

# Function to calculate and print torsions
def calculate_and_print_torsions(atoms):
    coords = [atom[1:] for atom in atoms]  # Extract coordinates
    n_atoms = len(atoms)
    torsions = []
    torsions = torsion_n  # Store torsions here
    connectivity = [[] for _ in range(n_atoms)]  # Bond connectivity matrix

    # Build connectivity matrix (bond list)
    for i in range(n_atoms):
        for j in range(i + 1, n_atoms):
            if bond_length(coords[i], coords[j]) < 1.6:  # Threshold for bonding
                connectivity[i].append(j)
                connectivity[j].append(i)

    # Find all torsions by iterating over atom pairs
    for j in range(n_atoms):
        for a in range(len(connectivity[j])):
            k = connectivity[j][a]
            if k < j:
                continue
            for b in range(len(connectivity[j])):
                i = connectivity[j][b]
                if i == k:
                    continue
                for c in range(len(connectivity[k])):
                    l = connectivity[k][c]
                    if l == j or l == i:
                        continue
                    # Calculate dihedral angle
                    angle_deg, angle_rad = dihedral_angle(coords[i], coords[j], coords[k], coords[l])

                    # Calculate torsion energy in degrees and radians
                    torsion_energy_deg = torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * np.radians(angle_deg)))
                    torsion_energy_rad = torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * angle_rad))

                    # Append torsion data (atom indices, angles, energies)
                    torsions.append((i, j, k, l, angle_deg, angle_rad, torsion_energy_deg, torsion_energy_rad))

    # Print torsions and total energies
    torsion_energy_total_deg = 0.0
    torsion_energy_total_rad = 0.0
    print("\nList of all torsion angles with energies:")

    for (i, j, k, l, angle_deg, angle_rad, torsion_energy_deg, torsion_energy_rad) in torsions:
        torsion_energy_total_deg += torsion_energy_deg
        torsion_energy_total_rad += torsion_energy_rad

        print(f"{atoms[i][0]}{i + 1} - {atoms[j][0]}{j + 1} - {atoms[k][0]}{k + 1} - {atoms[l][0]}{l + 1}: "
              f"{angle_deg:>10.3f} {angle_rad: >10.3f} {torsion_energy_deg:>20.6f}")

    # Print total torsion energy
    print(f"\nTotal torsion energy (calculated using degrees): {torsion_energy_total_deg:.6f} kcal/mol")
    print(f"Total torsion energy (calculated using radians): {torsion_energy_total_rad:.6f} kcal/mol")
    print(f"Number of torsions calculated: {len(torsions)}")

    # Append the energy to the list
    return torsion_energy_total_rad

# Example usage: Calculate and print torsions, also store the torsions
calculate_and_print_torsions(atoms)

print(torsion_n)



List of all torsion angles with energies:
H3 - C1 - C2 - H6:     56.162      0.980             0.006038
H3 - C1 - C2 - H7:    -66.111     -1.154             0.015228
H3 - C1 - C2 - H8:    177.400      3.096             0.002775
H4 - C1 - C2 - H6:   -177.164     -3.092             0.003302
H4 - C1 - C2 - H7:     60.563      1.057             0.000130
H4 - C1 - C2 - H8:    -55.925     -0.976             0.006802
H5 - C1 - C2 - H6:    -44.877     -0.783             0.089239
H5 - C1 - C2 - H7:   -167.150     -2.917             0.065381
H5 - C1 - C2 - H8:     76.362      1.333             0.103519

Total torsion energy (calculated using degrees): 0.292415 kcal/mol
Total torsion energy (calculated using radians): 0.292415 kcal/mol
Number of torsions calculated: 9
[(2, 0, 1, 5, 56.16168355850931, 0.980206291559153, 0.006038204041440065, 0.006038204041440065), (2, 0, 1, 6, -66.1112647804331, -1.1538592430763237, 0.015227966195526254, 0.015227966195526254), (2, 0, 1, 7, 177.40040831272125, 3.0

In [55]:
#internal coordinate

internal_coords = bond_count + angle_count +  len(torsion_n)
cartesian_coords = 3 * len(atoms)

print(f"Internal Coordinates: {internal_coords}")
print(f"Cartesian Coordinates: {cartesian_coords}")


Internal Coordinates: 28
Cartesian Coordinates: 24


In [56]:
Total_energy_vanderwaals = []

# Function to calculate van der Waals energy and print as it iterates
def calculate_and_print_vdw_energy(atoms, bonds_new, angles_new):
    # Define epsilon and sigma values for Hydrogen and Carbon
    epsilon_values = {'H': 0.03, 'C': 0.07}  # in kcal/mol
    sigma_values = {'H': 1.20, 'C': 1.75}    # in Angstroms

    total_energy = 0.0

    print(f"{'Atom Pair':<15} {'Distance (Å)':>15} {'Energy (kcal/mol)':>20}")

    N = len(atoms)

    # Iterate over all unique atom pairs
    for i in range(N):
        for j in range(i + 1, N):
            calc_vdw = True  # Assume we will calculate vdW for this pair
            # Exclude bonded pairs
            for bond in bonds_new:
                if i in bond and j in bond:
                    calc_vdw = False
                    break
            # Exclude pairs involved in angles
            if calc_vdw:
                for angle in angles_new:
                    if i in angle and j in angle:
                        calc_vdw = False
                        break

            # Retrieve atom types and coordinates
            atom1_type, x1, y1, z1 = atoms[i]
            atom2_type, x2, y2, z2 = atoms[j]
            r_ij = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

            if calc_vdw:
                # Retrieve epsilon and sigma values
                epsilon_i = epsilon_values.get(atom1_type)
                epsilon_j = epsilon_values.get(atom2_type)
                sigma_i = sigma_values.get(atom1_type)
                sigma_j = sigma_values.get(atom2_type)

                if epsilon_i is None or epsilon_j is None:
                    raise ValueError(f"Unknown atom type: {atom1_type} or {atom2_type}")

                # Compute mixed epsilon and sigma using geometric mean
                epsilon_ij = math.sqrt(epsilon_i * epsilon_j)
                sigma_ij = 2*math.sqrt(sigma_i * sigma_j)

                # Compute Lennard-Jones potential
                term12 = (sigma_ij / r_ij) ** 12
                term6 = (sigma_ij / r_ij) ** 6
                energy_lj = 4 * epsilon_ij * (term12 - term6)

                # Accumulate the total energy
                total_energy += energy_lj
                energy = energy_lj
            else:
                # For uncalculated pairs, energy is zero
                energy = 0.0

            # Print the current pair details
            print(f"{atom1_type} {i + 1:<3}- {atom2_type} {j + 1:<3}: {r_ij:>15.4f} {energy:>20.4f}")
            # Append the energy to the list
    Total_energy_vanderwaals.append(total_energy)

    print(f"\nTotal van der Waals Energy: {total_energy:.4f} kcal/mol")

# Example Input
# Calculate and print van der Waals energy
calculate_and_print_vdw_energy(atoms, bonds_new, angles_new)
print(bonds_new)



Atom Pair          Distance (Å)    Energy (kcal/mol)
C 1  - C 2  :          1.5536               0.0000
C 1  - H 3  :          1.0815               0.0000
C 1  - H 4  :          1.0684               0.0000
C 1  - H 5  :          1.2502               0.0000
C 1  - H 6  :          2.1402               0.0000
C 1  - H 7  :          2.1532               0.0000
C 1  - H 8  :          2.2179               0.0000
C 2  - H 3  :          2.2093               0.0000
C 2  - H 4  :          2.1798               0.0000
C 2  - H 5  :          2.3082               0.0000
C 2  - H 6  :          1.0176               0.0000
C 2  - H 7  :          1.1018               0.0000
C 2  - H 8  :          1.1384               0.0000
H 3  - H 4  :          1.7824               0.0000
H 3  - H 5  :          1.6785               0.0000
H 3  - H 6  :          2.5092              -0.0215
H 3  - H 7  :          2.5543              -0.0258
H 3  - H 8  :          3.1375              -0.0192
H 4  - H 5  :          1.9853

In [57]:
#Total_energy =   stretch_energy + bend_energy_radians + torsion_energy_total_rad + total_energy_vanderwaal
#print(Total_energy)

# New Section

In [58]:
#print_atom_coordinates(atoms)
#print("Number of coordinates: ")
#print(f"Stretching: {bond_count} Bending: {angle_count} Torsion: {len(torsion_n)}\nInternal: {internal_coords} Cartesian: {cartesian_coords}")
#print(f"Potential energy at input structure: {Energy_total: .6f}")


In [59]:


# Function to calculate the gradient (force) due to bond stretching
def bond_stretch_gradient(atoms, bond_params, bond_cutoffs):
    """
    Calculate the gradient (force) on each atom due to bond stretching.
    atoms: List of tuples (atom type, x, y, z) for each atom.
    bond_params: Dictionary of bond force constants (k_b) and equilibrium bond lengths (r_0).
    bond_cutoffs: Dictionary of bond cutoffs for detecting bonded atoms.
    """
    n_atoms = len(atoms)
    gradients = np.zeros((n_atoms, 3))  # Initialize gradient for each atom (3D vector)
    visited_bonds = set()  # To track counted bonds

    # Iterate over all pairs of atoms
    for i in range(n_atoms - 1):
        for j in range(i + 1, n_atoms):
            # Determine bond type dynamically
            bond_type = tuple(sorted((atoms[i][0], atoms[j][0])))

            # Only consider bonds defined in bond_params and bond_cutoffs
            if bond_type in bond_params and bond_type in bond_cutoffs:
                # Calculate the bond length
                r = bond_length(atoms[i][1:], atoms[j][1:])

                # Apply distance cutoffs to avoid counting non-bonded pairs
                min_cutoff, max_cutoff = bond_cutoffs[bond_type]
                if min_cutoff <= r <= max_cutoff and (i, j) not in visited_bonds:
                    visited_bonds.add((i, j))  # Mark this bond as visited

                    # Get bond parameters (force constant k_b and equilibrium bond length r_0)
                    k_b, r_0 = bond_params[bond_type]

                    # Compute the gradient of the bond energy with respect to atomic positions
                    # Bond length difference from equilibrium
                    delta_r = r_0-r

                    # Compute the gradient for both atoms i and j
                    # Gradient (force) on atom i
                    grad_i = -2 * k_b * delta_r * (np.array(atoms[i][1:]) - np.array(atoms[j][1:])) / r
                    # Gradient (force) on atom j
                    grad_j = 2 * k_b * delta_r * (np.array(atoms[i][1:]) - np.array(atoms[j][1:])) / r

                    # Update the gradients for atoms i and j
                    gradients[i] += grad_i
                    gradients[j] += grad_j

    return gradients





# Compute bond stretching gradient (forces)
gradients = bond_stretch_gradient(atoms, bond_params, bond_cutoffs)

# Output the gradients (forces) on each atom
print("Gradients (forces) on each atom:")
for i, grad in enumerate(gradients):
    print(f"Atom {i+1} (type {atoms[i][0]}): Force = {grad} kcal/mol/Å")


Gradients (forces) on each atom:
Atom 1 (type C): Force = [  6.66649805 102.29733846 -49.61452342] kcal/mol/Å
Atom 2 (type C): Force = [29.78841915 -4.91014381 71.08813205] kcal/mol/Å
Atom 3 (type H): Force = [  7.82543764 -11.03748108 -14.64964704] kcal/mol/Å
Atom 4 (type H): Force = [ 9.39854376 -8.46250993 26.2689844 ] kcal/mol/Å
Atom 5 (type H): Force = [-38.04179319 -82.34144172  37.53928033] kcal/mol/Å
Atom 6 (type H): Force = [-20.60968055  21.6071281  -57.34370427] kcal/mol/Å
Atom 7 (type H): Force = [-1.90396912 -5.32118888  1.08701228] kcal/mol/Å
Atom 8 (type H): Force = [  6.87654425 -11.83170114 -14.37553432] kcal/mol/Å


In [60]:
import numpy as np

# Function to calculate the bond length
def bond_length(atom_A, atom_B):
    """Compute the bond length between two atoms given their Cartesian coordinates."""
    return np.linalg.norm(np.array(atom_A) - np.array(atom_B))

# Normalize a vector
def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm != 0 else v

# Wilson B matrix calculation
def calculate_wilson_b_matrix(atoms):
    num_atoms = len(atoms)
    num_bonds = num_atoms - 1  # Stretching (C-H bonds)
    num_angles = (num_atoms - 1) * (num_atoms - 2) // 2  # Bending (H-C-H angles)

    num_internal_coords = num_bonds + num_angles  # Total internal coordinates
    b_matrix = np.zeros((num_internal_coords, 3 * num_atoms))  # B matrix

    internal_idx = 0

    # Stretching (C-H bonds)
    for i in range(1, num_atoms):
        bond = bond_length(atoms[0][1:], atoms[i][1:])  # Vector from C to H
        norm_bond = normalize(np.array(atoms[0][1:]) - np.array(atoms[i][1:]))  # Normalize bond vector
        b_matrix[internal_idx, 3*0:3*1] = -norm_bond  # Carbon contribution (negative sign)
        b_matrix[internal_idx, 3*i:3*(i+1)] = norm_bond  # Hydrogen contribution
        internal_idx += 1

    # Bending (H-C-H angles)
    for i in range(1, num_atoms - 1):
        for j in range(i + 1, num_atoms):
            bond1 = bond_length(atoms[0][1:], atoms[i][1:])
            bond2 = bond_length(atoms[0][1:], atoms[j][1:])
            norm_bond1 = normalize(np.array(atoms[0][1:]) - np.array(atoms[i][1:]))
            norm_bond2 = normalize(np.array(atoms[0][1:]) - np.array(atoms[j][1:]))
            cross_prod = np.cross(norm_bond1, norm_bond2)
            angle_term = np.linalg.norm(cross_prod)

            if angle_term != 0 and internal_idx < num_internal_coords:
                b_matrix[internal_idx, 3*0:3*1] = cross_prod / angle_term
                b_matrix[internal_idx, 3*i:3*(i+1)] = -cross_prod / angle_term
                b_matrix[internal_idx, 3*j:3*(j+1)] = -cross_prod / angle_term
                internal_idx += 1

    return b_matrix

# G matrix calculation (product of B and its transpose)
def calculate_g_matrix(b_matrix):
    g_matrix = np.dot(b_matrix, b_matrix.T)
    return g_matrix

# Print matrix with row numbering
def print_matrix_with_numbering(matrix, label):
    print(f"\n{label}:")
    for i, row in enumerate(matrix, start=1):
        row_str = ' '.join(f"{val:>10.5f}" for val in row)
        print(f"{i:>2}: {row_str}")

# Example atoms list (replace this with actual atom data)


# Calculate Wilson B matrix for the methane structure
b_matrix = calculate_wilson_b_matrix(atoms)

# Calculate the G matrix
g_matrix = calculate_g_matrix(b_matrix)

# Print the Wilson B matrix and G matrix with row numbering
print_matrix_with_numbering(b_matrix, "Wilson B Matrix at the initial structure")
print_matrix_with_numbering(g_matrix, "G matrix, the product of B with its own transpose (square, dimension 10)")



Wilson B Matrix at the initial structure:
 1:    0.99896   -0.03218    0.03218   -0.99896    0.03218   -0.03218    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000
 2:   -0.39241    0.55348    0.73462    0.00000    0.00000    0.00000    0.39241   -0.55348   -0.73462    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000
 3:   -0.32237    0.29026   -0.90102    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.32237   -0.29026    0.90102    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000
 4:   -0.38753   -0.83880    0.38241    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.00000    0.38753    0.8

In [61]:




B_matrix = compute_wilson_b_matrix(atoms, bonds_new, angles_new, torsion_n)
print("Wilson B Matrix:")
print(B_matrix)


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> None

In [66]:
import numpy as np

def bond_length(atom_A, atom_B):
    """Compute the bond length between two atoms given their Cartesian coordinates."""
    # Extract only the numeric coordinates (x, y, z) from atom data
    atom_A_coords = np.array(atom_A[1:])
    atom_B_coords = np.array(atom_B[1:])
    return np.linalg.norm(atom_A_coords - atom_B_coords)


def normalize(vector):
    # Normalize the vector
    norm = np.linalg.norm(vector)
    return vector / norm if norm != 0 else vector

def compute_bond_derivatives(atom_A, atom_B):
    # Compute derivatives of bond length with respect to Cartesian coordinates
    # Ensure that atom_A and atom_B are 3D vectors
    r_AB = np.array(atom_A) - np.array(atom_B)
    r_AB_norm = np.linalg.norm(r_AB)

    # Normalize the bond vector to get unit vector direction
    r_AB_unit = r_AB / r_AB_norm

    # Return derivatives of bond length with respect to each atom's coordinates
    return r_AB_unit, -r_AB_unit



def compute_angle_derivatives(atom_A, atom_B, atom_C):
    # Compute derivatives of bond angle with respect to Cartesian coordinates
    r_AB = np.array(atom_A) - np.array(atom_B)
    r_BC = np.array(atom_C) - np.array(atom_B)
    p = np.cross(r_AB, r_BC)
    r_AB_norm = np.linalg.norm(r_AB)
    r_BC_norm = np.linalg.norm(r_BC)

    if np.linalg.norm(p) == 0:
        return None, None, None

    d_theta_B = (np.cross(r_AB, p) / r_AB_norm**2).reshape(1, 3) + (np.cross(-r_BC, p) / r_BC_norm**2).reshape(1, 3)
    d_theta_A = -np.cross(r_AB, p) / r_AB_norm**2
    d_theta_C = np.cross(r_BC, p) / r_BC_norm**2

    return d_theta_B.flatten(), d_theta_A.flatten(), d_theta_C.flatten()

def compute_dihedral_derivatives(atom_A, atom_B, atom_C, atom_D):
    # Compute derivatives of dihedral angle with respect to Cartesian coordinates
    r_AB = np.array(atom_A) - np.array(atom_B)
    r_BC = np.array(atom_B) - np.array(atom_C)
    r_CD = np.array(atom_C) - np.array(atom_D)

    t = np.cross(r_AB, r_BC)
    u = np.cross(r_BC, r_CD)

    if np.linalg.norm(t) == 0 or np.linalg.norm(u) == 0:
        return None, None, None, None

    # Dihedral derivatives with respect to A, B, C, D
    d_phi_A = np.cross(t, r_BC) / np.linalg.norm(t)**2
    d_phi_B = np.cross(t, r_AB) / np.linalg.norm(t)**2 + np.cross(-u, r_BC) / np.linalg.norm(u)**2
    d_phi_C = np.cross(t, r_BC) / np.linalg.norm(t)**2 - np.cross(-u, r_BC) / np.linalg.norm(u)**2
    d_phi_D = np.cross(-u, r_CD) / np.linalg.norm(u)**2

    return d_phi_A.flatten(), d_phi_B.flatten(), d_phi_C.flatten(), d_phi_D.flatten()

def compute_wilson_b_matrix(atoms, bonds, angles, dihedrals):
    # Corrected line to compute nq
    nq = len(bonds) + len(angles) + len(dihedrals)  # Internal coordinates count
    nx = len(atoms) * 3  # Cartesian coordinates count
    B_matrix = np.zeros((nq, nx))

    internal_idx = 0

# Bond length derivatives
    for i, (atom_A_idx, atom_B_idx) in enumerate(bonds):
        atom_A = atoms[atom_A_idx]
        atom_B = atoms[atom_B_idx]
        bond_derivatives = compute_bond_derivatives(atom_A[1:], atom_B[1:])  # Extract coordinates only
        B_matrix[internal_idx, 3*atom_A_idx:3*(atom_A_idx+1)] = bond_derivatives[0]
        B_matrix[internal_idx, 3*atom_B_idx:3*(atom_B_idx+1)] = bond_derivatives[1]
        internal_idx += 1


    # Angle derivatives
    for i, (atom_A_idx, atom_B_idx, atom_C_idx) in enumerate(angles):
        atom_A = atoms[atom_A_idx]
        atom_B = atoms[atom_B_idx]
        atom_C = atoms[atom_C_idx]
        angle_derivatives = compute_angle_derivatives(atom_A, atom_B, atom_C)
        B_matrix[internal_idx, 3*atom_A_idx:3*(atom_A_idx+1)] = angle_derivatives[1]
        B_matrix[internal_idx, 3*atom_B_idx:3*(atom_B_idx+1)] = angle_derivatives[0]
        B_matrix[internal_idx, 3*atom_C_idx:3*(atom_C_idx+1)] = angle_derivatives[2]
        internal_idx += 1

    # Dihedral derivatives
    for i, (atom_A_idx, atom_B_idx, atom_C_idx, atom_D_idx) in enumerate(dihedrals):
        atom_A = atoms[atom_A_idx]
        atom_B = atoms[atom_B_idx]
        atom_C = atoms[atom_C_idx]
        atom_D = atoms[atom_D_idx]
        dihedral_derivatives = compute_dihedral_derivatives(atom_A, atom_B, atom_C, atom_D)
        B_matrix[internal_idx, 3*atom_A_idx:3*(atom_A_idx+1)] = dihedral_derivatives[0]
        B_matrix[internal_idx, 3*atom_B_idx:3*(atom_B_idx+1)] = dihedral_derivatives[1]
        B_matrix[internal_idx, 3*atom_C_idx:3*(atom_C_idx+1)] = dihedral_derivatives[2]
        B_matrix[internal_idx, 3*atom_D_idx:3*(atom_D_idx+1)] = dihedral_derivatives[3]
        internal_idx += 1

    return B_matrix
# Assuming atoms, bonds_new, angles_new, and torsion_n are already defined
B_matrix = compute_wilson_b_matrix(atoms, bonds_new, angles_new, torsion_n)
print("Wilson B Matrix:")
print(B_matrix)


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> None